In [ ]:
# 一、Pytorch基本操作考察（平台课+专业课）

In [ ]:
# 1.
# 使用 𝐓𝐞𝐧𝐬𝐨𝐫 初始化一个 𝟏 × 𝟑 的矩阵 𝑴 和一个 𝟐 × 𝟏 的矩阵 𝑵，对两矩阵进行减法操作（要求实现三种不同的形式），给出结果并分析三种方式的不同（如果出现报错，分析报错的原因），同时需要指出在计算过程中发生了什么

In [2]:
import torch

In [4]:
M = torch.tensor([[1], [2], [3]])
N = torch.tensor([4, 5])
Q = M - N
print(Q)

tensor([[-3, -4],
        [-2, -3],
        [-1, -2]])


In [9]:
M_2 = torch.rand_like(M, dtype=torch.float)
N_2 = torch.rand_like(N, dtype=torch.float)
Q_2 = M - N
print(Q_2)

tensor([[-3, -4],
        [-2, -3],
        [-1, -2]])


In [10]:
M_3 = torch.ones(1, 3)
N_3 = torch.ones(2, 1)
Q_3 = M_3 - N_3
print(Q_3)

tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [21]:
print("减法1 = {}".format(M - N))
print("剑法2 = {}".format(M.subtract(N)))
print("剑法3 = {}".format(M.sub_(N)))

减法1 = tensor([[-3, -4],
        [-2, -3],
        [-1, -2]])
剑法2 = tensor([[-3, -4],
        [-2, -3],
        [-1, -2]])


RuntimeError: output with shape [3, 1] doesn't match the broadcast shape [3, 2]

In [ ]:
# 前两种减法会形成广播，两个矩阵均变为3 x 2形状，第三种不形成广播所以提示形状报错

In [ ]:
# 2.
# ① 利用 𝐓𝐞𝐧𝐬𝐨𝐫 创建两个大小分别 𝟑 × 𝟐 和 𝟒 × 𝟐 的随机数矩阵 𝑷 和 𝑸 ，要求服从均值为0，标准差0.01为的正态分布

In [11]:
P = torch.normal(0, 0.01, (3, 2))
Q = torch.normal(0, 0.01, (4, 2))
print(P)
print(Q)

tensor([[ 0.0015, -0.0076],
        [ 0.0172,  0.0250],
        [ 0.0125, -0.0004]])
tensor([[ 0.0008,  0.0104],
        [-0.0054, -0.0046],
        [-0.0199,  0.0153],
        [-0.0008, -0.0010]])


In [ ]:
# ② 对第二步得到的矩阵 𝑸 进行形状变换得到 𝑸 的转置 𝑸𝑻

In [12]:
QT = Q.T
print(QT)

tensor([[ 0.0008, -0.0054, -0.0199, -0.0008],
        [ 0.0104, -0.0046,  0.0153, -0.0010]])


In [ ]:
# ③ 对上述得到的矩阵 𝑷 和矩阵 𝑸𝑻 求矩阵相乘

In [14]:
PQT = torch.mm(P, QT)
print(PQT)

tensor([[-7.8270e-05,  2.7024e-05, -1.4622e-04,  6.4623e-06],
        [ 2.7337e-04, -2.0765e-04,  4.1129e-05, -3.8444e-05],
        [ 5.4841e-06, -6.5841e-05, -2.5488e-04, -9.4149e-06]])


In [ ]:
# 3.
# 给定公式 𝑦3 = 𝑦1 + 𝑦2 = 𝑥^2 + 𝑥^3，且 𝑥 = 1。利用学习所得到的Tensor的相关知识，求𝑦3对𝑥的梯度，即𝑑𝑦3/𝑑𝑥
# 要求在计算过程中，在计算 𝑥^3 时中断梯度的追踪，观察结果并进行原因分析
# 提示, 可使用 with torch.no_grad()， 举例:
# with torch.no_grad():
# y = x * 5

In [29]:
x = torch.tensor([1.0], dtype=float, requires_grad=True)
y1 = x * x
with torch.no_grad():
    y2 = x * x * x
y3 = y1 + y2
y3.backward()
print(x.grad)

tensor([2.], dtype=torch.float64)


In [31]:
# 二、动手实现 logistic 回归（平台课+专业课）

In [32]:
# 1. 要求动手从0实现 logistic 回归（只借助Tensor和Numpy相关的库）在人工构造的数据集上进行训练和测试，并从loss以及训练集上的准确率等多个角度对结果进行分析
# （可借助nn.BCELoss或nn.BCEWithLogitsLoss作为损失函数，从零实现二元交叉熵为选作）

In [ ]:
# 定义生成数据集的绘图函数
from matplotlib import pyplot as plt
import matplotlib_inline.backend_inline

def set_figsize(ipython_format, figsize=(3.5, 2.5)):
    matplotlib_inline.backend_inline.set_matplotlib_formats(ipython_format)
    plt.rcParams['figure.figsize'] = figsize

# set_figsize('svg')
# plt.scatter(features[:, 1].numpy(), labels.numpy(), 1)

In [ ]:
# 定义损失与正确率的绘图函数
def figplot(fignum, loss, acc):
    plt.figure(figsize=(8,3))
    plt.suptitle('Figure ' + str(fignum))
    # 打印损失值
    plt.subplot(121)
    plt.ylabel('Loss')
    plt.plot(loss[0],label='Train Loss')
    plt.plot(loss[1],label='Test Loss')
    plt.legend()
    # 打印正确率
    plt.subplot(122)
    plt.ylabel('ACC')
    plt.plot(acc[0],label='Train Acc')
    plt.plot(acc[1],label='Test Acc')
    plt.legend()
    plt.show()

In [71]:
# 人工构造数据集函数
import torch
import numpy as np

def generate_data(num_examples, num_inputs, true_w, true_b):
    features = torch.tensor(np.random.rand(num_examples, num_inputs), dtype=torch.float)
    labels = 1 / (1 + torch.exp(-1 * (true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b)))
    labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)
    result_0 = 0
    result_1 = 0
    for i in range(num_examples):
        if labels[i] < 0.5:
            labels[i] = 0
            result_0 += 1
        else:
            labels[i] = 1
            result_1 += 1
    return features, labels, result_0, result_1

In [74]:
# 生成训练数据集与测试数据集并进行可视化展示
num_inputs = 2
true_w = [2, -3.4]
true_b = 0.7
train_examples,test_examples = 2000, 1000
train_data,train_labels, train_0, train_1 = generate_data(train_examples,num_inputs, true_w, true_b)
test_data, test_labels, test_0, test_1 = generate_data(test_examples,num_inputs, true_w, true_b)
print("训练集共有数据%d个，其中标签为'0'的数量为 %d， 标签为'1'的数量为 %d"%(train_examples,train_0,train_1))
print("测试集共有数据%d个，其中标签为'0'的数量为 %d， 标签为'1'的数量为 %d"%(test_examples,test_0,test_1))

训练集共有数据2000个，其中标签为'0'的数量为 1000， 标签为'1'的数量为 1000
测试集共有数据1000个，其中标签为'0'的数量为 465， 标签为'1'的数量为 535


In [75]:
# 定义数据读取函数
import random
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices) # 样本的读取顺序是随机的
    for i in range(0, num_examples, batch_size):
        j = torch.LongTensor(indices[i: min(i + batch_size, num_examples)]) # 最后一次可能不足一个batch
        yield features.index_select(0, j), labels.index_select(0, j)

In [76]:
# 构建logistic模型
w = torch.tensor(np.random.normal(0, 0.01, (num_inputs, 1)), dtype=torch.float32)
b = torch.zeros(1, dtype=torch.float32)
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)
def logistic(X, w, b):
    logistic_func = 1 / (1 + torch.exp(-1 * (torch.mm(X, w) + b)))
    return logistic_func

In [77]:
# 定义优化函数
def sgd(params, lr, batch_size):
    for param in params:
        param.data -= lr * param.grad / batch_size # 注意这里更改param时用的param.data

In [81]:
# 定义损失函数
from torch.nn import BCELoss

In [106]:
# 模型训练
lr = 0.6
num_epochs = 100
batch_size = 64
net = logistic
loss = BCELoss()
train_all_loss = []
acc_all = []
for epoch in range(num_epochs): # 训练模型一共需要num_epochs个迭代周期
    # 在每一个迭代周期中，会使用训练数据集中所有样本一次
    for X, y in data_iter(batch_size, train_data, train_labels): # x和y分别是小批量样本的特征和标签
        pred = net(X, w, b)
        y = y.view(-1, 1)
        train_each_loss = loss(pred, y)
        train_each_loss.backward() # 反向传播
        sgd([w,b], lr, batch_size) # 使用小批量随机梯度下降迭代模型参数
        # 梯度清零
        w.grad.data.zero_()
        b.grad.data.zero_()
    labels_pred = net(train_data, w, b)
    train_loss = loss(labels_pred, train_labels.view(-1, 1))
    train_all_loss.append(train_loss.item())
    labels_pred = torch.tensor(np.where(labels_pred<0.5, 0, 1),dtype=torch.float32)
    # acc = (labels_pred==train_labels).sum(0).item() / train_examples
    acc = len((labels_pred==train_labels).sum(0))
    # max_acc = max(acc,max_acc)
    # acc_all.append(acc)
    # if epoch==0 or (epoch+1) % 10 == 0:
    #     print('epoch: %d loss:%.5f acc: %.3f'%(epoch+1, train_loss.item(), acc))
    if epoch==0 or (epoch+1) % 10 == 0:
        print('epoch: %d loss:%.5f '%(epoch+1, train_loss.item()))

2000
epoch: 1 loss:0.15019 
2000
2000
2000
2000
2000
2000
2000
2000
2000
epoch: 10 loss:0.14990 
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
epoch: 20 loss:0.14959 
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
epoch: 30 loss:0.14928 
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
epoch: 40 loss:0.14896 
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
epoch: 50 loss:0.14866 
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
epoch: 60 loss:0.14835 
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
epoch: 70 loss:0.14805 
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
epoch: 80 loss:0.14775 
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
epoch: 90 loss:0.14745 
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
epoch: 100 loss:0.14715 
